## Create Financial Valuation Using Discounted Cash Flow Analysis

## References

1. [Investopedia](http://i.investopedia.com/inv/pdf/tutorials/dcf.pdf)
2. [...](https://.com)

## Equation

$$ DCF = \frac{FCFF_1}{(1 + r) ^ 1} + \frac{FCFF_2}{(1 + r) ^ 2} + ... + \frac{FCFF_n}{(1 + r) ^ n} $$

Where $FCFF$ stands for unlevered free cash flow for a given year ($FCFF_1$ means free cash flow in year 1, and so on); and $r$ stands for the discount rate.

$ FCFF = EBITDA - CAPEX - \Delta NWC - tax\_expenses $

Where:
1. EBITDA = earnings before interest, depreciation, and amortization, or pretax income,
2. CAPEX = capital expenditure, or fixed assets,
3. $ \Delta $ NWC = changes in net working capital, which can be derived by subtracting total current assets and total current liabilities in current and previous years,
4. tax_expenses is already self-explanatory (hopefully).

## Load Data

In [1]:
import os
import pandas
import requests
import string
from functools import partial, reduce
from bs4 import BeautifulSoup

In [2]:
PATH = os.getcwd() + '/sample-output/'

In [42]:
def get_single_dataframe(year):
    
    file_name = [x for x in os.listdir('sample-output/') if x.endswith('{}.csv'.format(year))]
    
    dfs = []
    for f in file_name:
        dfs.append(pandas.read_csv(PATH + f))
        
    return reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

In [43]:
def get_multiple_dataframes(year):
    
    dfs = []
    for year in (year - 1, year):
        dfs.append(get_dataframe(year))
        
    return pandas.concat(dfs, sort = False).reset_index(drop = True)

In [44]:
df = get_multiple_dataframes(2018)

## Calculate Free Cash Flow

In [45]:
def get_free_cash_flow(year):
    
    # select necessary variables
    # sort by ticker code in ascending order
    # reset index
    df_nwc = df[['year', 'ticker_code', 'total_current_assets', 'total_current_liabilities']]\
    .sort_values(by=['ticker_code'], ascending=True)\
    .reset_index(drop=True)
    
    # add net working capital as a column
    df_nwc['net_working_capital'] = df_nwc['total_current_assets'] - df_nwc['total_current_liabilities']
    
    # changes in net working capital
    df_nwc['net_working_capital_delta'] = df_nwc.groupby('ticker_code')['net_working_capital'].diff()
    
    # filter dataframe only in year latest year
    df_nwc = df_nwc[df_nwc['year'] == year][['ticker_code', 'net_working_capital_delta']].reset_index(drop=True)
    
    # calculate free cash flow
    df_cf = df[df['year'] == 2018][['year', 'ticker_code', 'pretax_income', 'fixed_assets', 'tax_expenses']]
    df_cf = pandas.merge(df_cf, df_nwc, how = 'inner', on = 'ticker_code').reset_index(drop = True)
    df_cf['free_cash_flow'] = df_cf['pretax_income'] \
    - df_cf['fixed_assets'] \
    - df_cf['net_working_capital_delta'] \
    - df_cf['tax_expenses']
    
    return df_cf[['year', 'ticker_code', 'free_cash_flow']].reset_index(drop=True)

In [46]:
df_cf = get_free_cash_flow(2018)

## Calculate Discount Rate (r)

What we need to find out:
1. Cost of Equity 
2. Cost of Debt

### Cost of Equity

Cost of equity, or RE, can obtained by calculating [Capital Asset Pricing Model](https://www.investopedia.com/terms/c/capm.asp). Following are the items:

1. Risk-free rate
2. Beta
3. Equity market risk premium (market rate subtracted by risk-free rate)

After getting the items, we need to calculate the cost of equity as follows:

$$ RE = risk\_free\_rate + beta * (market\_rate - risk\_free\_rate) $$

### Risk-free rate

Risk-free rate is commonly derived from T-bills rate. In Indonesia, the equivalent is [Obligasi Negara](https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx), which is traded in the secondary market. Currently there are many traded bonds with various coupon rates, so I am going to use the average rate. First, we need to scrape the table to get the coupon rate. 

In [58]:
def get_risk_free_rate():
    
    # get bonds rate from Bank Central of Indonesia
    # attempt thrice in case of connection failure
    i = 0
    while i < 3:
        try:
            url = requests.get('https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx')
            if url.status_code == 200:
                soup = BeautifulSoup(url.text, 'html')
            i = 3
        except Exception:
            i += 1
    
    # find table which contains the bonds rate
    table = soup.find('table', {'class': 'table1'})
    trs = table.find_all('tr')
    
    # get the bonds rate
    tds = []
    for t in trs:
        try:
            tds.append(t.find_all('td')[-1].text.replace(' ', ''))
        except IndexError:
            pass
        
    for t in range(len(tds)):
        if '\r' in tds[t]:
            tds[t] = tds[t].replace('\r', '')
        elif '\n' in tds[t]:
            tds[t] = tds[t].replace('\n', '')
        try:
            tds[t] = float(tds[t]) / 100
        except ValueError:
            pass
        
    risk_free_rate = []
    for t in tds:
        if type(t) == float:
            risk_free_rate.append(t)
    
    # returns bonds rate
    return sum(risk_free_rate) / len(risk_free_rate)

In [49]:
def get_cost_of_equity(risk_free_rate, beta, market_rate):
    
    return risk_free_rate + beta * (market_rate - risk_free_rate)

### Cost of Debt

Cost of debt, or symbolized as RD, can be calculated as follows:

$$ RD = 1 - tax\_rate $$

According to [Trading Economics](https://tradingeconomics.com/indonesia/corporate-tax-rate), Indonesia's corporate tax rate is 25%. Hence, the cost of equity is 75%.

### Capital Structure (Weighted Average Cost of Capital)

In [ ]:
def get_wacc():

    # get capital structure
    df_wacc = df[df['year'] == 2018][['ticker_code', 'total_assets', 'total_equity']].reset_index(drop = True)
    
    df_wacc['equity_proportion'] = df_wacc['total_equity'] / df_wacc['total_assets']
    df_wacc['liabilities_proportion'] = 1 - df_wacc['equity_proportion']
    
    # cost of equity
    re = get_cost_of_equity(
        risk_free_rate=get_risk_free_rate(),
        beta=1,
        market_rate=0.05
    )
    
    # cost of debt
    rd = 1 - 0.25
    
    # weighted cost of capital
    df_wacc['wacc'] = (re * df_wacc['equity_proportion']) + (rd * (1 - df_wacc['equity_proportion']))
    df_wacc = df_wacc[['ticker_code', 'wacc']]
    
    return df_wacc

In [ ]:
df_wacc = get_wacc()

In [60]:
df = pandas.merge(df_cf, df_wacc, how = 'inner', on = 'ticker_code')

## Calculate Fair Value

What we need to find out is the terminal value

$$ terminal\_value = \frac{FPYCF * (1 + LTCFGR)}{r - LTCFGR} $$

Jeez, what's that?!?!?!

1. FPYCF = Final projected year cash flow
2. LTCGR = Long-term cash flow growth rate
3. r = discounted rate

In [79]:
def get_fair_value(rate, num_year):
    
    df['terminal_value'] = (df['free_cash_flow'] * (1 + (rate * num_year)) \
                            * (1 + rate) \
                            / (df['wacc'] - rate))
    
    
    df['enterprise_value'] = df['free_cash_flow'] / ((1 + rate) ** num_year)
    
    df['fair_value'] = df['terminal_value'] - df['enterprise_value']
    
    return df

In [81]:
df_fv = get_fair_value(0.01, 1)

In [84]:
df_fv[df_fv['fair_value'] >= 0].reset_index(drop = True)[['year', 'ticker_code', 'fair_value']]

,year,ticker_code,fair_value
0,2018,APOL,1.205725e+11
1,2018,BIMA,4.946429e+09
2,2018,CMPP,3.733407e+10
3,2018,CNKO,1.195950e+08
4,2018,CSIS,1.822183e+11
5,2018,EMTK,2.298630e+09
6,2018,ENRG,1.784610e+07
7,2018,ERAA,1.898704e+12
8,2018,ETWA,5.821560e+10
9,2018,FORU,8.478821e+09
